<a href="https://colab.research.google.com/github/ziyaad30/vits/blob/main/vits_trainer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Please note this is only setup for using my Google drive. Will be updated soon!

In [ ]:
!nvidia-smi

In [ ]:
!git clone https://github.com/ziyaad30/vits.git

In [ ]:
%cd vits

In [ ]:
!pip install -r requirements.txt

In [ ]:
#@title Install custom PyTorch
!pip install torch==1.13.1+cu117 torchvision==0.14.1+cu117 torchaudio==0.13.1 --extra-index-url https://download.pytorch.org/whl/cu117

In [ ]:
%cd monotonic_align

In [ ]:
!mkdir monotonic_align

In [ ]:
!python setup.py build_ext --inplace

In [ ]:
%cd ..

In [ ]:
!sudo apt-get install espeak -y # Doubt this is needed for training

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#@title Train
%reload_ext tensorboard
%tensorboard --logdir /content/drive/MyDrive/vits-files/logs
!python train.py -c configs/ljs_base.json -m /content/drive/MyDrive/vits-files/logs

In [ ]:
#@title Synthesis
%matplotlib inline
import matplotlib.pyplot as plt
import IPython.display as ipd

import os
import json
import math
import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import DataLoader

import commons
import utils
from data_utils import TextAudioLoader, TextAudioCollate, TextAudioSpeakerLoader, TextAudioSpeakerCollate
from models import SynthesizerTrn
from text.symbols import symbols
from text import text_to_sequence

from scipy.io.wavfile import write


def get_text(text, hps):
    text_norm = text_to_sequence(text, hps.data.text_cleaners)
    if hps.data.add_blank:
        text_norm = commons.intersperse(text_norm, 0)
    text_norm = torch.LongTensor(text_norm)
    return text_norm

In [ ]:
hps = utils.get_hparams_from_file("./configs/ljs_base.json")

In [ ]:
net_g = SynthesizerTrn(
    len(symbols),
    hps.data.filter_length // 2 + 1,
    hps.train.segment_size // hps.data.hop_length,
    **hps.model).cpu()
_ = net_g.eval()

_ = utils.load_checkpoint("/content/drive/MyDrive/vits-files/logs/G_8000.pth", net_g, None)

In [ ]:
stn_tst = get_text("this really is not working at all my friend. But, let's see how it goes tomorrow. Have a great day further!", hps) # TODO: split sentences.
with torch.no_grad():
    x_tst = stn_tst.cpu().unsqueeze(0)
    x_tst_lengths = torch.LongTensor([stn_tst.size(0)]).cpu()
    audio = net_g.infer(x_tst, x_tst_lengths, noise_scale=.667, noise_scale_w=0.8, length_scale=1)[0][0,0].data.cpu().float().numpy()
ipd.display(ipd.Audio(audio, rate=hps.data.sampling_rate, normalize=True))